In [ ]:
import numpy as np
import pandas as pd
import torch

In [ ]:
train_df = pd.read_csv("data/input/train.csv")
test_df = pd.read_csv("data/input/test.csv")
print(train_df.shape , test_df.shape)
print("Train Cols : " , train_df.columns)
print("Test Cols : " , test_df.columns)
print(" Label -  " , [i for i in train_df.columns if i not in test_df.columns] )

In [ ]:
train_df.head()

In [ ]:
train_df['label'].value_counts().sort_index()

In [ ]:
# Convert df to numpy array
images = np.array(train_df.iloc[:,1:])

# Reshape the array to 32000x784 to 32000x28x28
images = images.reshape(-1,28,28)

In [ ]:
images.shape

In [ ]:
# Scaling the image so that the values only range between 0 and 1
images = images/255.0

# Converting to tensor
images = torch.from_numpy(images)
images = images.to(torch.float32)

In [ ]:
# Convert labels to one-hot encoding tensor
labels = np.array(train_df['label'])#.reshape(-1,1)
labels.shape

In [ ]:
image_label = np.zeros((images.shape[0], 10))
image_label[np.arange(images.shape[0]), labels] = 1.0

In [ ]:
image_label = torch.from_numpy(image_label)
image_label = image_label.to(torch.float32)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(images, image_label, test_size=0.1, stratify=train_df['label'])


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
# Ploting some of the datapoints in the dataset
import matplotlib.pyplot as plt

# sample_img , sample_lbl = temp_train_dataset[3]
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
figure.add_subplot(rows, cols, 1)
plt.axis("off")
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(X_train), size=(1,)).item()
    sample_img = X_train[sample_idx]
    sample_lbl = y_train[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(sample_lbl)
    plt.axis("off")
    plt.imshow(sample_img.squeeze(), cmap="gray")
plt.show()

In [ ]:
## Checking if the GPU is being used properly.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)


In [ ]:
from torch import nn

# Softmax layer ignored since the loss function defined is nn.CrossEntropy()
class MyOwnNeuralNetwork(nn.Module):
    def __init__(self):
        super(MyOwnNeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return  logits


model = MyOwnNeuralNetwork().to(device)
print(model)

model = model.cuda()
torch.backends.cudnn.benchmark=True
torch.cuda.set_device(0)

In [ ]:
## Defining optimizer and loss functions
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=3e-3, momentum=0.9)

In [ ]:
def pytorch_train(dataloader, model, loss_fn, optimizer):
    model.train()

    size = 0
    correct = 0

    batch_loss = np.zeros(len(dataloader))
    batch_accuracy = np.zeros(len(dataloader))

    for batch, (X, y) in enumerate(dataloader):
        # Feedforward / Loss
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy
        _correct = (pred.argmax(1) == y.argmax(1)).type(torch.float32).sum().item()

        # Updating loss_batch and batch_accuracy
        batch_loss[batch] = loss.item()
        batch_accuracy[batch] = _correct/X.shape[0]

        # Updating size and correct
        size += X.shape[0]
        correct += _correct

        if batch % 100 == 0:
            loss = loss.item()
            current = batch * X.shape[0]
            print(f"loss: {loss:>7f}  [{current:>5d}]")

    accuracy = correct / size
    print(f"Train Accuracy: {(100*accuracy):>0.1f}%")

    return batch_loss, batch_accuracy


In [ ]:
def pytorch_validation(dataloader, model, loss_fn):
    model.eval()

    size = 0
    correct = 0

    batch_loss = np.zeros(len(dataloader))
    batch_accuracy = np.zeros(len(dataloader))
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            # Feedforward / Loss
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)

            # Accuracy
            _correct = (pred.argmax(1) == y.argmax(1)).type(torch.float32).sum().item()

            # Updating loss_batch and batch_accuracy
            batch_loss[batch] = loss.item()
            batch_accuracy[batch] = _correct/X.shape[0]

            # Updating size and correct
            size += X.shape[0]
            correct += _correct

    accuracy = correct / size
    print(f"Validation Accuracy: {(100*accuracy):>0.1f}%")

    return batch_loss, batch_accuracy

In [ ]:
# Batch the data
batch_size = 64

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
train_batch_loss = []
train_batch_accuracy = []
valid_batch_accuracy = []
valid_batch_loss = []
train_epoch_no = []
valid_epoch_no = []

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    _train_batch_loss , _train_batch_accuracy = pytorch_train(train_dataloader, model, loss_fn, optimizer)
    _valid_batch_loss , _valid_batch_accuracy = pytorch_validation(val_dataloader, model, loss_fn)

    for i in range(len(_train_batch_loss)):
        train_batch_loss.append(_train_batch_loss[i])
        train_batch_accuracy.append(_train_batch_accuracy[i])
        train_epoch_no.append( t + float((i+1)/len(_train_batch_loss)))

    for i in range(len(_valid_batch_loss)):
        valid_batch_loss.append(_valid_batch_loss[i])
        valid_batch_accuracy.append(_valid_batch_accuracy[i])
        valid_epoch_no.append( t + float((i+1)/len(_valid_batch_loss)))
print("Done!")

In [ ]:
figure = plt.figure(figsize=(16, 16))


figure.add_subplot(2, 2, 1)
plt.plot(train_epoch_no , train_batch_accuracy)
plt.title("Train Batch Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Train Accuracy")

figure.add_subplot(2, 2, 2)
plt.plot(train_epoch_no , train_batch_loss)
plt.title("Train Batch Loss")
plt.xlabel("Epochs")
plt.ylabel("Train Loss")

figure.add_subplot(2, 2, 3)
plt.plot(valid_epoch_no , valid_batch_accuracy)
plt.title("Valid Batch Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Train Accuracy")

figure.add_subplot(2, 2, 4)
plt.plot(valid_epoch_no , valid_batch_loss)
plt.title("Valid Batch Loss")
plt.xlabel("Epochs")
plt.ylabel("Train Loss")


plt.show()